In [2]:
pip install selenium

     ---------------------------------------- 9.7/9.7 MB 1.4 MB/s eta 0:00:00
     -------------------------------------- 481.7/481.7 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 58.8/58.8 kB 1.0 MB/s eta 0:00:00
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
     ---------------------------------------- 58.3/58.3 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.2.0
    Uninstalling sniffio-1.2.0:
      Successfully uninstalled sniffio-1.2.0
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Uninstalling attrs-21.4.0:
      Successfully uninstalled attrs-21.4.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# URL to scrape
url = 'https://letterboxd.com/films/popular/year/1990/size/large/'

# Set up Selenium WebDriver (Ensure you have downloaded the appropriate WebDriver)
driver = webdriver.Chrome()  # Replace with webdriver.Firefox() if using Firefox

# Open the webpage
driver.get(url)

# Wait for the page to load fully (adjust time as needed)
time.sleep(5)

# Get the page source after JavaScript rendering
html_content = driver.page_source

# Close the browser
driver.quit()

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all <li> tags with the class "listitem poster-container"
list_items = soup.find_all("li", {"class": "listitem poster-container"})

# Extract and print movie names and poster URLs
for item in list_items:
    # Extract movie name from the 'data-film-name' attribute
    movie_name = item.find('span', class_ ="frame-title" )
    movie_name = movie_name.text
    
    # Extract poster URL from the <img> tag's 'src' attribute
    img_tag = item.find("img")
    poster_url = img_tag['src'] if img_tag else "No poster URL found"
    
    print(f"Movie: {movie_name}, Poster URL: {poster_url}")


Movie: GoodFellas (1990), Poster URL: https://a.ltrbxd.com/resized/film-poster/5/1/3/8/3/51383-goodfellas-0-150-0-225-crop.jpg?v=c6c265f228
Movie: Home Alone (1990), Poster URL: https://a.ltrbxd.com/resized/sm/upload/e6/mn/oi/ky/8IWPBT1rkAaI8Kpk5V3WfQRklJ7-0-150-0-225-crop.jpg?v=cb22d36a0e
Movie: Edward Scissorhands (1990), Poster URL: https://a.ltrbxd.com/resized/film-poster/5/1/8/8/9/51889-edward-scissorhands-0-150-0-225-crop.jpg?v=38f2599419
Movie: Pretty Woman (1990), Poster URL: https://a.ltrbxd.com/resized/film-poster/5/1/9/3/6/51936-pretty-woman-0-150-0-225-crop.jpg?v=b190c1d6bc
Movie: Back to the Future Part III (1990), Poster URL: https://a.ltrbxd.com/resized/film-poster/5/1/8/5/6/51856-back-to-the-future-part-iii-0-150-0-225-crop.jpg?v=fdbcbb74cb
Movie: Misery (1990), Poster URL: https://a.ltrbxd.com/resized/film-poster/5/0/8/4/6/50846-misery-0-150-0-225-crop.jpg?v=f079fc8138
Movie: The Godfather Part III (1990), Poster URL: https://a.ltrbxd.com/resized/film-poster/5/1/8/1/4/

In [6]:
import requests

In [160]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def wait_for_page_load(driver, timeout=10):
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CLASS_NAME, "listitem poster-container"))
    )

from concurrent.futures import ThreadPoolExecutor

def parallel_scrape_movie_details(movie_urls):
    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust the number of workers as needed
        results = list(executor.map(scrape_movie_details, movie_urls))
    return results

    

In [168]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas
import sys


# Function to scrape details from a movie page
def scrape_movie_details(movie_url):
    driver.get(movie_url)  # Navigate to the movie page
    time.sleep(3)  # Wait for the page to load

    # Get the page source and parse it
    movie_soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract movie details
    movie_name = movie_soup.find('span', class_ = 'name js-widont prettify').text.strip().replace('\xa0', ' ') if movie_soup.find("h1") else "No title found"
    release_year = movie_soup.find('div', class_ = 'releaseyear').find('a').text.strip() if movie_soup.find('div', class_ = 'releaseyear') else "Movie release year not found"
    runtime_tag = movie_soup.find("p", class_="text-link text-footer").text.strip() 
    runtime_tag = runtime_tag.strip().split(' ')[0]+'mins' if runtime_tag else "No runtime found"
    runtime = runtime_tag[0:3]+' mins'
    rating = movie_soup.find("meta", {"name": "twitter:data2"})['content'] if movie_soup.find("meta", {"name": "twitter:data2"}) else "No rating found"
    director_tag = movie_soup.find('p', class_ = 'credits').find('span', class_ = 'directorlist').find('a', class_ = 'contributor')
    director = director_tag.find('span', class_ = 'prettify').text.strip() if director_tag else "No director found"
    description = movie_soup.find("div", class_ = 'review body-text -prose -hero prettify').find("div", class_ = 'truncate').text.strip().replace('\xa0', ' ') if movie_soup.find("div", class_ = 'review body-text -prose -hero prettify') else "No description found"

    # Cast, Genre, Country, and Language
#     cast_tags = [a.text.strip() for a in movie_soup.find('div', id_ = 'tabbed-content').find('div', id_ = 'tab-cast', class_ = 'tabbed-content-block')] 
#     cast = cast_tags.find('div', class_ = 'cast-list text-sluglist').find_all('a', class_ = 'text-slug tooltip').text.strip() if cast_tags else "No cast found"
    cast_tags = movie_soup.find('div', class_='cast-list text-sluglist')
    if cast_tags:
        cast = [a.text.strip() for a in cast_tags.find_all('a', class_='text-slug tooltip')]
        main_cast = cast[0:10]
    else:
        cast = "No cast found"
    genre = [a.text.strip() for a in movie_soup.find_all("a", {"href": lambda x: x and "/genre/" in x})] if movie_soup.find_all("a", {"href": lambda x: x and "/genre/" in x}) else "No genre found"
    country = movie_soup.find("a", {"href": lambda x: x and "/films/country/" in x}).text.strip() if movie_soup.find("a", {"href": lambda x: x and "/films/country/" in x}) else "No country found"
    language = movie_soup.find("a", {"href": lambda x: x and "/films/language/" in x}).text.strip() if movie_soup.find("a", {"href": lambda x: x and "/films/language/" in x}) else "No language found"

    return {
        "Movie Name": movie_name,
        "Movie Release Year": release_year,
        "Runtime": runtime,
        "Rating": rating,
        "Director": director,
        "Description": description,
        "Cast": main_cast,
        "Genre": genre,
        "Country": country,
        "Language": language
    }

# Creating the empty list
movies_list = []

# looping over the years in given range 
for year in range(1990, 2025):
    # URL to scrape
    url = f'https://letterboxd.com/films/popular/year/{year}/size/large/'

    # Set up Selenium WebDriver
    driver = webdriver.Chrome()  
    driver.set_page_load_timeout(300)

    # Open the main webpage
    driver.get(url)

    # Wait for the page to load fully
    time.sleep(5)

    # Get the page source after JavaScript rendering
    html_content = driver.page_source

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <li> tags with the class "listitem poster-container"
    list_items = soup.find_all("li", {"class": "listitem poster-container"})

    # Extract and print details for each movie
    for item in list_items:  # Extended to get all movies
        # Extract movie link
        a_tag = item.find("a")
        movie_url = f"https://letterboxd.com{a_tag['href']}" if a_tag else None

        if movie_url:
            # Scrape details from the movie page
            details = scrape_movie_details(movie_url)
            movies_list.append(details)
            sys.stdout.write(f"\rNumber of movies scraped: {len(movies_list)}")
            sys.stdout.flush()
    #         print(details)

movies_df= pd.DataFrame(movies_list)
movies_df
# Close the browser
driver.quit()


Number of movies scraped: 630

In [169]:
movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
0,GoodFellas,1990,145 mins,4.47 out of 5,Martin Scorsese,"The true story of Henry Hill, a half-Irish, ha...","[Robert De Niro, Ray Liotta, Joe Pesci, Lorrai...","[Drama, Crime]",USA,English
1,Home Alone,1990,103 mins,3.81 out of 5,Chris Columbus,Eight-year-old Kevin McCallister makes the mos...,"[Macaulay Culkin, Joe Pesci, Daniel Stern, Cat...","[Family, Comedy]",USA,English
2,Edward Scissorhands,1990,105 mins,3.88 out of 5,Tim Burton,A small suburban town receives a visit from a ...,"[Johnny Depp, Winona Ryder, Dianne Wiest, Anth...","[Drama, Romance, Fantasy]",USA,English
3,Pretty Woman,1990,120 mins,3.65 out of 5,Garry Marshall,"While on a business trip in Los Angeles, Edwar...","[Richard Gere, Julia Roberts, Jason Alexander,...","[Romance, Comedy]",USA,English
4,Back to the Future Part III,1990,119 mins,3.54 out of 5,Robert Zemeckis,The final installment of the Back to the Futur...,"[Michael J. Fox, Christopher Lloyd, Mary Steen...","[Science Fiction, Comedy, Adventure]",USA,English
...,...,...,...,...,...,...,...,...,...,...
625,Trap,2024,105 mins,2.72 out of 5,M. Night Shyamalan,A father and teen daughter attend a pop concer...,"[Josh Hartnett, Ariel Donoghue, Saleka, Alison...","[Crime, Thriller]",USA,English
626,I Saw the TV Glow,2024,100 mins,3.53 out of 5,Jane Schoenbrun,Teenager Owen is just trying to make it throug...,"[Justice Smith, Brigette Lundy-Paine, Ian Fore...","[Drama, Horror]",USA,English
627,Love Lies Bleeding,2024,104 mins,3.57 out of 5,Rose Glass,Reclusive gym manager Lou falls hard for Jacki...,"[Kristen Stewart, Katy O'Brian, Ed Harris, Dav...","[Romance, Thriller, Crime]",UK,English
628,A Quiet Place: Day One,2024,99 mins,3.19 out of 5,Michael Sarnoski,As New York City is invaded by alien creatures...,"[Lupita Nyong'o, Joseph Quinn, Alex Wolff, Dji...","[Thriller, Science Fiction, Horror]",USA,English


In [170]:
movies_df.index = range(1,len(movies_df)+1)
movies_df

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
1,GoodFellas,1990,145 mins,4.47 out of 5,Martin Scorsese,"The true story of Henry Hill, a half-Irish, ha...","[Robert De Niro, Ray Liotta, Joe Pesci, Lorrai...","[Drama, Crime]",USA,English
2,Home Alone,1990,103 mins,3.81 out of 5,Chris Columbus,Eight-year-old Kevin McCallister makes the mos...,"[Macaulay Culkin, Joe Pesci, Daniel Stern, Cat...","[Family, Comedy]",USA,English
3,Edward Scissorhands,1990,105 mins,3.88 out of 5,Tim Burton,A small suburban town receives a visit from a ...,"[Johnny Depp, Winona Ryder, Dianne Wiest, Anth...","[Drama, Romance, Fantasy]",USA,English
4,Pretty Woman,1990,120 mins,3.65 out of 5,Garry Marshall,"While on a business trip in Los Angeles, Edwar...","[Richard Gere, Julia Roberts, Jason Alexander,...","[Romance, Comedy]",USA,English
5,Back to the Future Part III,1990,119 mins,3.54 out of 5,Robert Zemeckis,The final installment of the Back to the Futur...,"[Michael J. Fox, Christopher Lloyd, Mary Steen...","[Science Fiction, Comedy, Adventure]",USA,English
...,...,...,...,...,...,...,...,...,...,...
626,Trap,2024,105 mins,2.72 out of 5,M. Night Shyamalan,A father and teen daughter attend a pop concer...,"[Josh Hartnett, Ariel Donoghue, Saleka, Alison...","[Crime, Thriller]",USA,English
627,I Saw the TV Glow,2024,100 mins,3.53 out of 5,Jane Schoenbrun,Teenager Owen is just trying to make it throug...,"[Justice Smith, Brigette Lundy-Paine, Ian Fore...","[Drama, Horror]",USA,English
628,Love Lies Bleeding,2024,104 mins,3.57 out of 5,Rose Glass,Reclusive gym manager Lou falls hard for Jacki...,"[Kristen Stewart, Katy O'Brian, Ed Harris, Dav...","[Romance, Thriller, Crime]",UK,English
629,A Quiet Place: Day One,2024,99 mins,3.19 out of 5,Michael Sarnoski,As New York City is invaded by alien creatures...,"[Lupita Nyong'o, Joseph Quinn, Alex Wolff, Dji...","[Thriller, Science Fiction, Horror]",USA,English


In [175]:
movies_df.to_csv("Hollywood_movies_dataset", index=False)

In [176]:
pd.read_csv("Hollywood_movies_dataset")

,Movie Name,Movie Release Year,Runtime,Rating,Director,Description,Cast,Genre,Country,Language
0,GoodFellas,1990,145 mins,4.47 out of 5,Martin Scorsese,"The true story of Henry Hill, a half-Irish, ha...","['Robert De Niro', 'Ray Liotta', 'Joe Pesci', ...","['Drama', 'Crime']",USA,English
1,Home Alone,1990,103 mins,3.81 out of 5,Chris Columbus,Eight-year-old Kevin McCallister makes the mos...,"['Macaulay Culkin', 'Joe Pesci', 'Daniel Stern...","['Family', 'Comedy']",USA,English
2,Edward Scissorhands,1990,105 mins,3.88 out of 5,Tim Burton,A small suburban town receives a visit from a ...,"['Johnny Depp', 'Winona Ryder', 'Dianne Wiest'...","['Drama', 'Romance', 'Fantasy']",USA,English
3,Pretty Woman,1990,120 mins,3.65 out of 5,Garry Marshall,"While on a business trip in Los Angeles, Edwar...","['Richard Gere', 'Julia Roberts', 'Jason Alexa...","['Romance', 'Comedy']",USA,English
4,Back to the Future Part III,1990,119 mins,3.54 out of 5,Robert Zemeckis,The final installment of the Back to the Futur...,"['Michael J. Fox', 'Christopher Lloyd', 'Mary ...","['Science Fiction', 'Comedy', 'Adventure']",USA,English
...,...,...,...,...,...,...,...,...,...,...
625,Trap,2024,105 mins,2.72 out of 5,M. Night Shyamalan,A father and teen daughter attend a pop concer...,"['Josh Hartnett', 'Ariel Donoghue', 'Saleka', ...","['Crime', 'Thriller']",USA,English
626,I Saw the TV Glow,2024,100 mins,3.53 out of 5,Jane Schoenbrun,Teenager Owen is just trying to make it throug...,"['Justice Smith', 'Brigette Lundy-Paine', 'Ian...","['Drama', 'Horror']",USA,English
627,Love Lies Bleeding,2024,104 mins,3.57 out of 5,Rose Glass,Reclusive gym manager Lou falls hard for Jacki...,"['Kristen Stewart', ""Katy O'Brian"", 'Ed Harris...","['Romance', 'Thriller', 'Crime']",UK,English
628,A Quiet Place: Day One,2024,99 mins,3.19 out of 5,Michael Sarnoski,As New York City is invaded by alien creatures...,"[""Lupita Nyong'o"", 'Joseph Quinn', 'Alex Wolff...","['Thriller', 'Science Fiction', 'Horror']",USA,English
